In [1]:
from Tool_Functions.cleaning_data import *
from Tool_Functions.comportment_reabo import * 
from Tool_Functions.join_data import * 

In [2]:
path_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/"
path_results_antoine = "/Users/antoine/Documents/ENSAE2A/Codeperso/everything/Statappperso/Ressources/Results/"

Nouveaux indicateurs : 
- Mois du réabonnement 
- Delai de réabonnement moyen pour cette promo comparé au délai de réabonnement moyen pour toutes les promos
- Trouver aussi un moyen de quantifier le poids de l’abonnement par rapport aux habitudes de fidélité d’un client : s’il s’abonne alors qu’il s’abonne jamais, la promo aura un poids plus important

In [3]:
df = file_to_dataframe(path_antoine + "df_Donnees_Reabos.csv")

In [10]:
dfp = file_to_dataframe(path_antoine + "df_Donnees_Promos.csv")

In [4]:
odd = file_to_dataframe(path_antoine + "df_Donnees_Reabos_odd.csv")

In [9]:
oddp = file_to_dataframe(path_antoine + "df_Donnees_Promos_odd.csv")

In [17]:
oddp.shape

(11415480, 5)

In [21]:
dfp

,Unnamed: 0,ID_ABONNE,DATE_ACTE_REEL,DATE_DEMARRAGE_PROMO,CPROMO
0,0,51478366401,2021-06-16,2021-06-16,SGEVP01
1,1,52006231501,2021-06-16,2021-06-16,SGEVP01
2,2,51349585101,2021-06-16,2021-06-16,SGEVP01
3,3,52108880501,2021-06-16,2021-06-16,SGEVP01
4,4,51947630401,2021-06-16,2021-06-16,SGEVP01
...,...,...,...,...,...
15733966,15733966,51974732901,2023-05-01,2023-05-01,SGEVP01
15733967,15733967,51923691301,2023-05-01,2023-05-01,SGEVP01
15733968,15733968,50993144501,2023-05-01,2023-05-01,SGEVP01
15733969,15733969,52103629101,2023-05-01,2023-05-01,SGEVP01


•⁠  ⁠transformer la fidélité en un score et essayer de le croiser avec les types de promo
•⁠  ⁠⁠comparer les délais de réabonnements à ce qu’il se fait hors promotion 
•⁠  ⁠⁠on peut dans un premier temps se concentrer sur un seul type d’ODD (ex 15j TC)

créer la table sans ODD

In [114]:
df = file_to_dataframe(path_antoine + 'new_datas_diff_outer.csv')

In [115]:
df

,ID_ABONNE,MOY_DELAI,Autres_MEAN_TIME_DIFF,ODD 15 jours EV+_MEAN_TIME_DIFF,ODD 15 jours TC_MEAN_TIME_DIFF,ODD 21 jours TC_MEAN_TIME_DIFF,ODD 30 jours EV+_MEAN_TIME_DIFF,ODD 30 jours TC_MEAN_TIME_DIFF,ODD 7 jours autre que SG_MEAN_TIME_DIFF,Semaine genéreuse_MEAN_TIME_DIFF,noODD_MEAN_TIME_DIFF
0,50402221602,-2.666667,NaN,NaN,0.666667,NaN,NaN,-4.333333,NaN,NaN,3.666667
1,50402241202,-4.296333,NaN,NaN,1.629333,NaN,NaN,NaN,NaN,-4.925667,3.296333
2,50402254602,-3.875000,NaN,NaN,2.275000,NaN,NaN,0.375000,NaN,-3.525000,0.875000
3,50402266702,9.222333,NaN,NaN,8.444667,NaN,NaN,NaN,NaN,0.777667,-9.222333
4,50402282802,-0.625000,NaN,NaN,0.125000,NaN,NaN,0.625000,NaN,-2.375000,1.625000
...,...,...,...,...,...,...,...,...,...,...,...
270957,53166085401,-45.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.000000,0.000000
270958,53174006601,-25.166667,NaN,-4.833333,NaN,NaN,NaN,NaN,NaN,9.666667,-4.833333
270959,53174006601,-25.166667,NaN,-4.833333,NaN,NaN,NaN,NaN,NaN,9.666667,-4.833333
270960,53175791201,-3.000000,NaN,-2.000000,-2.000000,NaN,NaN,NaN,NaN,2.000000,2.000000
